# Learnable Alpha Pipeline Runner

This notebook walks through executing the end-to-end learnable alpha workflow directly from Python, logging each shell command and summarising the generated artefacts.

## 1. Validate Current Working Directory
Confirm the notebook is pointed at the `MKA_improvement` folder before launching the pipeline commands.

In [ ]:
from pathlib import Path

PROJECT_ROOT = Path.cwd().resolve()
print(f"Working directory: {PROJECT_ROOT}")

required_paths = [
    "pipeline.py",
    "evaluate_methods.py",
    "plot_results.py",
    "data",
    "models",
    "experiments",
]
print("\nKey entries:")
for entry in required_paths:
    status = "exists" if (PROJECT_ROOT / entry).exists() else "missing"
    print(f" - {entry}: {status}")

if PROJECT_ROOT.name != "MKA_improvement":
    print("\n[!] Consider changing directories so this notebook runs inside MKA_improvement/")

## 2. Helper to Run and Log Shell Commands
The helper below streams stdout/stderr so long-running jobs (e.g., LLM training) remain observable inside the notebook.

In [ ]:
import subprocess
import sys
from datetime import datetime

EXECUTE_COMMANDS = False  # Flip to True to actually run the heavy CLI commands.


def run_command(command: str, *, cwd: Path | None = None) -> int:
    """Execute a shell command, streaming output line by line."""
    print(f"\n$ {command}")
    if not EXECUTE_COMMANDS:
        print("[dry-run] EXECUTE_COMMANDS is False; command not executed.")
        return 0

    process = subprocess.Popen(
        command,
        shell=True,
        cwd=str(cwd) if cwd else None,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1,
        universal_newlines=True,
    )

    for line in process.stdout:
        timestamp = datetime.now().strftime("%H:%M:%S")
        sys.stdout.write(f"[{timestamp}] {line}")

    return_code = process.wait()
    if return_code != 0:
        raise RuntimeError(f"Command failed with exit code {return_code}:")
    print(f"\nCommand completed successfully (exit code {return_code}).")
    return return_code

## 3. Launch Training Pipeline Command
This cell runs the main MKA pipeline with learnable alpha enabled. Update paths or hyperparameters as needed for your environment before executing.

In [ ]:
pipeline_command = (
    'python pipeline.py '
    '--model_path "meta-llama/Meta-Llama-3-8B" '
    '--num_layer 14 '
    '--data_dir "./data" '
    '--use_learnable_alpha '
    '--alpha_training_steps 500 '
    '--alpha_learning_rate 1e-4'
)

print("Pipeline command:")
print(pipeline_command)
run_command(pipeline_command, cwd=PROJECT_ROOT)

## 4. Run MMLU Evaluation Command
Evaluates the fused model on MMLU. Adjust the arguments if your evaluation script expects different paths (e.g., `--output_dir`).

In [ ]:
evaluation_command = (
    'python evaluate_methods.py '
    '--model_path "./merged_weights/learnable_alpha" '
    '--output_file "results.json"'
)

print("Evaluation command:")
print(evaluation_command)
run_command(evaluation_command, cwd=PROJECT_ROOT)

## 5. Generate Result Plots
This command should populate the `./plots` directory with the visual summaries of the evaluation.

In [ ]:
plot_command = (
    'python plot_results.py '
    '--results_file "results.json" '
    '--output_dir "./plots"'
)

print("Plotting command:")
print(plot_command)
run_command(plot_command, cwd=PROJECT_ROOT)

## 6. Inspect Generated Artefacts and Logs
After the commands finish, summarise the outputs to confirm artefacts were created.

In [ ]:
from itertools import chain

paths_to_check = [
    PROJECT_ROOT / "merged_weights",
    PROJECT_ROOT / "results.json",
    PROJECT_ROOT / "plots",
]

for path in paths_to_check:
    print(f"\nInspecting: {path}")
    if not path.exists():
        print(" - [missing]")
        continue
    if path.is_file():
        size_kb = path.stat().st_size / 1024
        print(f" - file size: {size_kb:.2f} KB")
        with path.open("r", encoding="utf-8", errors="ignore") as handle:
            snippet = ''.join(handle.readlines()[:5])
        print(" - preview:\n" + snippet)
    else:
        entries = list(path.iterdir())
        if not entries:
            print(" - directory is empty")
        else:
            for entry in entries:
                kind = "dir" if entry.is_dir() else "file"
                print(f" - ({kind}) {entry.name}")

---

## Alternative: MLP-Based Dynamic Merging

Instead of a scalar α, use an MLP that predicts α based on input statistics.

In [ ]:
mlp_pipeline_command = (
    'python pipeline.py '
    '--model_path "meta-llama/Meta-Llama-3-8B" '
    '--num_layer 14 '
    '--data_dir "./data" '
    '--use_learnable_alpha '
    '--use_mlp_merge '
    '--alpha_training_steps 500 '
    '--alpha_learning_rate 1e-4'
)

print("MLP-based merging command:")
print(mlp_pipeline_command)
run_command(mlp_pipeline_command, cwd=PROJECT_ROOT)

## Comparing Both Methods

Run full evaluation suite including both scalar α and MLP α variants.

In [ ]:
full_eval_command = (
    'python evaluate_methods.py '
    '--model_path "meta-llama/Meta-Llama-3-8B" '
    '--data_dir "./data" '
    '--similarity_matrix "similarity_matrix.pkl" '
    '--output_dir "./experiments" '
    '--include_mlp'
)

print("Full evaluation suite (includes MLP):")
print(full_eval_command)
run_command(full_eval_command, cwd=PROJECT_ROOT)